## *** Setup Spark***

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
!tar -zxvf spark-3.3.2-bin-hadoop3.tgz

spark-3.3.2-bin-hadoop3/
spark-3.3.2-bin-hadoop3/LICENSE
spark-3.3.2-bin-hadoop3/NOTICE
spark-3.3.2-bin-hadoop3/R/
spark-3.3.2-bin-hadoop3/R/lib/
spark-3.3.2-bin-hadoop3/R/lib/SparkR/
spark-3.3.2-bin-hadoop3/R/lib/SparkR/DESCRIPTION
spark-3.3.2-bin-hadoop3/R/lib/SparkR/INDEX
spark-3.3.2-bin-hadoop3/R/lib/SparkR/Meta/
spark-3.3.2-bin-hadoop3/R/lib/SparkR/Meta/Rd.rds
spark-3.3.2-bin-hadoop3/R/lib/SparkR/Meta/features.rds
spark-3.3.2-bin-hadoop3/R/lib/SparkR/Meta/hsearch.rds
spark-3.3.2-bin-hadoop3/R/lib/SparkR/Meta/links.rds
spark-3.3.2-bin-hadoop3/R/lib/SparkR/Meta/nsInfo.rds
spark-3.3.2-bin-hadoop3/R/lib/SparkR/Meta/package.rds
spark-3.3.2-bin-hadoop3/R/lib/SparkR/Meta/vignette.rds
spark-3.3.2-bin-hadoop3/R/lib/SparkR/NAMESPACE
spark-3.3.2-bin-hadoop3/R/lib/SparkR/R/
spark-3.3.2-bin-hadoop3/R/lib/SparkR/R/SparkR
spark-3.3.2-bin-hadoop3/R/lib/SparkR/R/SparkR.rdb
spark-3.3.2-bin-hadoop3/R/lib/SparkR/R/SparkR.rdx
spark-3.3.2-bin-hadoop3/R/lib/SparkR/doc/
spark-3.3.2-bin-hadoop3/R/lib/Spar

In [2]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [972 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2,060 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,324 kB]
Hit:13 http://ppa.launchpad.net/graphics-dri

In [3]:
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=76652a3f491488c7404051da517552e34fb26885fbed0ca9a8640007f1098255
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

In [5]:
import pyspark
spark = pyspark.sql.SparkSession.builder.appName("zomato-recommendation").getOrCreate()
sc = spark.sparkContext

In [6]:
#Importing Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [67]:
data= spark.read.csv('/gdrive/My Drive/Big Data Framework/zomato.csv', inferSchema=True, header=True)

In [70]:
data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+--------------------+--------------------+--------------------+--------------------+
|                 url|             address|                name|        online_order|          book_table| rate|               votes|               phone|            location|           rest_type|          dish_liked|            cuisines|approx_cost(for two people)|        reviews_list|           menu_item|     listed_in(type)|     listed_in(city)|
+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------------------+--------------------+-----------

In [71]:
data.columns


['url',
 'address',
 'name',
 'online_order',
 'book_table',
 'rate',
 'votes',
 'phone',
 'location',
 'rest_type',
 'dish_liked',
 'cuisines',
 'approx_cost(for two people)',
 'reviews_list',
 'menu_item',
 'listed_in(type)',
 'listed_in(city)']

In [72]:
data.printSchema()


root
 |-- url: string (nullable = true)
 |-- address: string (nullable = true)
 |-- name: string (nullable = true)
 |-- online_order: string (nullable = true)
 |-- book_table: string (nullable = true)
 |-- rate: string (nullable = true)
 |-- votes: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- location: string (nullable = true)
 |-- rest_type: string (nullable = true)
 |-- dish_liked: string (nullable = true)
 |-- cuisines: string (nullable = true)
 |-- approx_cost(for two people): string (nullable = true)
 |-- reviews_list: string (nullable = true)
 |-- menu_item: string (nullable = true)
 |-- listed_in(type): string (nullable = true)
 |-- listed_in(city): string (nullable = true)



In [73]:
#Deleting Unnnecessary Columns
from pyspark.sql.functions import col
zomato=data.drop("url", "dish_liked", "phone")

In [74]:
from pyspark.sql.functions import count, when
count_all = zomato.count()
count_distinct = zomato.distinct().count()
count_duplicates = count_all - count_distinct
print(count_duplicates)
zomato = zomato.dropDuplicates()

25720


In [75]:
from pyspark.sql.functions import col, count
null_counts = zomato.agg(*[count(col(c)).alias(c) for c in zomato.columns])
null_counts.show()
zomato = zomato.dropna(how='any')
zomato.printSchema()

+-------+-----+------------+----------+-----+-----+--------+---------+--------+---------------------------+------------+---------+---------------+---------------+
|address| name|online_order|book_table| rate|votes|location|rest_type|cuisines|approx_cost(for two people)|reviews_list|menu_item|listed_in(type)|listed_in(city)|
+-------+-----+------------+----------+-----+-----+--------+---------+--------+---------------------------+------------+---------+---------------+---------------+
|  46010|45942|       39587|     46008|39666|46010|   37248|    37152|   30146|                      29552|       29390|    28967|          28595|          28241|
+-------+-----+------------+----------+-----+-----+--------+---------+--------+---------------------------+------------+---------+---------------+---------------+

root
 |-- address: string (nullable = true)
 |-- name: string (nullable = true)
 |-- online_order: string (nullable = true)
 |-- book_table: string (nullable = true)
 |-- rate: string 

In [76]:
zomato.columns

['address',
 'name',
 'online_order',
 'book_table',
 'rate',
 'votes',
 'location',
 'rest_type',
 'cuisines',
 'approx_cost(for two people)',
 'reviews_list',
 'menu_item',
 'listed_in(type)',
 'listed_in(city)']

In [77]:
zomato = zomato.withColumnRenamed("approx_cost(for two people)", "cost") \
               .withColumnRenamed("listed_in(type)", "type") \
               .withColumnRenamed("listed_in(city)", "city")

print(zomato.columns)

['address', 'name', 'online_order', 'book_table', 'rate', 'votes', 'location', 'rest_type', 'cuisines', 'cost', 'reviews_list', 'menu_item', 'type', 'city']


In [78]:
from pyspark.sql.functions import  regexp_replace
zomato = zomato.withColumn("cost", col("cost").cast("string"))
zomato = zomato.withColumn("cost", regexp_replace(col("cost"), ",", "."))
zomato = zomato.withColumn("cost", col("cost").cast("float"))
zomato.printSchema()

root
 |-- address: string (nullable = true)
 |-- name: string (nullable = true)
 |-- online_order: string (nullable = true)
 |-- book_table: string (nullable = true)
 |-- rate: string (nullable = true)
 |-- votes: string (nullable = true)
 |-- location: string (nullable = true)
 |-- rest_type: string (nullable = true)
 |-- cuisines: string (nullable = true)
 |-- cost: float (nullable = true)
 |-- reviews_list: string (nullable = true)
 |-- menu_item: string (nullable = true)
 |-- type: string (nullable = true)
 |-- city: string (nullable = true)



In [79]:
zomato.select("rate").distinct().show()

+------+
|  rate|
+------+
|   800|
| 3.8/5|
|   700|
|   200|
| 2,000|
| 1,400|
| 2.2/5|
|   250|
| 2,100|
|4.0 /5|
| 4.9/5|
| 1,700|
| 1,800|
|4.9 /5|
| 1,600|
| 1,100|
| 3.3/5|
| 2.4/5|
| 4.2/5|
|3.5 /5|
+------+
only showing top 20 rows



In [80]:
zomato = zomato.filter(col("rate") != "NEW")
zomato = zomato.filter(col("rate") != "-").na.drop()
zomato = zomato.withColumn("rate", regexp_replace(col("rate"), "/5", ""))
zomato = zomato.withColumn("rate", col("rate").cast("float"))
zomato.select("rate").show()

+----+
|rate|
+----+
| 3.4|
| 3.4|
| 3.6|
| 3.9|
| 4.2|
| 3.6|
| 2.7|
| 3.3|
| 3.7|
| 4.1|
| 4.0|
| 3.9|
| 3.8|
| 4.0|
| 3.0|
| 3.9|
| 4.1|
| 3.3|
| 4.4|
| 3.7|
+----+
only showing top 20 rows



In [81]:
from pyspark.sql.functions import col, initcap

zomato = zomato.withColumn("name", initcap(col("name")))
zomato = zomato.withColumn("online_order", when(col("online_order") == "Yes", True).otherwise(False))
zomato = zomato.withColumn("book_table", when(col("book_table") == "Yes", True).otherwise(False))
zomato.select("cost").distinct().show()

+-----+
| cost|
+-----+
|550.0|
|500.0|
|180.0|
|  2.5|
|350.0|
|  2.2|
|  3.4|
|100.0|
|  2.0|
|  1.8|
|  3.2|
| 1.45|
|  3.0|
| 70.0|
|  1.5|
| 40.0|
|250.0|
|  1.1|
|  2.8|
|400.0|
+-----+
only showing top 20 rows



In [55]:
zomato.show(20)

+--------------------+--------------------+------------+----------+----+-----+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|             address|                name|online_order|book_table|rate|votes|            location|           rest_type|            cuisines| cost|        reviews_list|           menu_item|                type|                city|
+--------------------+--------------------+------------+----------+----+-----+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|715, Hari Complex...|Karavali Fish Center|       false|     false| 3.4|   10|        Banashankari|         Quick Bites|Mangalorean, Chinese|300.0|"[('Rated 4.0', '...| This place serve...|        ('Rated 3.0'| 'RATED\n  Ambien...|
|Shop 984, Btm 2nd...|          Lassi Berg|        true|     false| 3.4|

In [56]:
zomato.select("location").distinct().show()

+--------------------+
|            location|
+--------------------+
|           Bellandur|
|      East Bangalore|
|         Indiranagar|
|                 BTM|
|        Banashankari|
|Koramangala 7th B...|
|            JP Nagar|
|        Lavelle Road|
|        Kammanahalli|
|Koramangala 3rd B...|
|Koramangala 2nd B...|
|      St. Marks Road|
|            Majestic|
|ITPL Main Road, W...|
|           Jayanagar|
|        Brigade Road|
|     Electronic City|
|         Frazer Town|
|       Church Street|
|                 HSR|
+--------------------+
only showing top 20 rows



In [83]:
from pyspark.sql.functions import mean
from pyspark.sql.window import Window
from pyspark.sql.functions import col

window = Window.partitionBy("name")
zomato = zomato.withColumn("Mean Rating", mean(col("rate")).over(window))



In [84]:
zomato.show(10)

+--------------------+--------------------+------------+----------+----+-----+--------------------+-------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+------------------+
|             address|                name|online_order|book_table|rate|votes|            location|    rest_type|            cuisines| cost|        reviews_list|           menu_item|                type|                city|       Mean Rating|
+--------------------+--------------------+------------+----------+----+-----+--------------------+-------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+------------------+
|Sector 6, HSR Lay...|          #l-81 Cafe|        true|     false| 3.9|   48|                 HSR|  Quick Bites|Fast Food, Beverages|400.0|"[('Rated 4.0', '...| just came here b...| especially comfo...|        ('Rated 5.0'|3.9000000953674316|
|Sector 6, HSR Lay...|  

In [86]:
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from pyspark.ml.linalg import VectorUDT
to_vector = udf(lambda x: Vectors.dense(x), VectorUDT())

zomato = zomato.withColumn("Mean Rating", to_vector(col("Mean Rating")))

In [91]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType

vector_to_double = udf(lambda x: float(x[0]), DoubleType())
scaler = MinMaxScaler(inputCol="Mean Rating", outputCol="Scaled Mean Rating", min=1, max=5)
scalerModel = scaler.fit(zomato)
zomato = scalerModel.transform(zomato)
zomato = zomato.withColumn("Scaled Mean Rating", vector_to_double(col("Scaled Mean Rating")))
zomato = zomato.withColumn("Scaled Mean Rating", F.round(col("Scaled Mean Rating"), 2))
zomato.sample(0.1).show(3)


+--------------------+---------------+------------+----------+----+-----+--------------------+-------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+
|             address|           name|online_order|book_table|rate|votes|            location|    rest_type|            cuisines| cost|        reviews_list|           menu_item|                type|                city|        Mean Rating|Scaled Mean Rating|
+--------------------+---------------+------------+----------+----+-----+--------------------+-------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+
|3, 80 Feet Road, ...| 1522 - The Pub|       false|      true| 4.2| 1743|Koramangala 4th B...|          Pub|Chinese, Continen...|  1.4|"[('Rated 5.0', '...| peanuts and chic...|        ('Rated 4.0'| 'RATED\n  A good...|[4.1

In [92]:
zomato.select('reviews_list', 'cuisines').sample(False, 0.05)

DataFrame[reviews_list: string, cuisines: string]

In [93]:
from pyspark.sql.functions import lower

zomato = zomato.withColumn("reviews_list", lower(zomato["reviews_list"]))
zomato.select('reviews_list', 'cuisines').sample(False, 0.05)

DataFrame[reviews_list: string, cuisines: string]

In [94]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import string

PUNCT_TO_REMOVE = string.punctuation

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

remove_punctuation_udf = udf(remove_punctuation, StringType())

zomato = zomato.withColumn("reviews_list", remove_punctuation_udf(zomato["reviews_list"]))
zomato.select('reviews_list', 'cuisines').sample(False, 0.05)

DataFrame[reviews_list: string, cuisines: string]

In [96]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [99]:
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType
from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

remove_stopwords_udf = udf(remove_stopwords, StringType())

split_udf = udf(lambda x: x.split(), ArrayType(StringType()))

zomato = zomato.withColumn('reviews_list_split', split_udf('reviews_list'))

remover = StopWordsRemover(inputCol="reviews_list_split", outputCol="reviews_list_filtered", stopWords=list(STOPWORDS))
zomato = remover.transform(zomato)

zomato = zomato.withColumn("reviews_list", remove_stopwords_udf(zomato["reviews_list_filtered"])) \
             .drop("reviews_list_filtered", "reviews_list_split")



In [100]:
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

remove_urls_udf = udf(remove_urls, StringType())

zomato = zomato.withColumn("reviews_list", remove_urls_udf(zomato["reviews_list"]))

In [101]:
zomato.select('reviews_list', 'cuisines').sample(False, 0.05)

DataFrame[reviews_list: string, cuisines: string]

In [104]:
from pyspark.ml.feature import CountVectorizer
from pyspark.sql.functions import udf,lit,array
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, IntegerType

def get_top_words(column, top_nu_of_words, nu_of_word):
    vec = CountVectorizer(ngram_range=nu_of_word, stopWords='english')
    bag_of_words = vec.fit(column).transform(column)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:top_nu_of_words]

get_top_words_udf = udf(get_top_words, ArrayType(StructType([
    StructField("word", StringType(), True),
    StructField("frequency", IntegerType(), True)
])))

zomato = zomato.withColumn("top_words", get_top_words_udf(zomato["reviews_list"], lit(10), array(lit(1), lit(2))))

In [113]:
from pyspark.sql.functions import col
zomato = zomato.drop(*['address', 'rest_type', 'type', 'menu_item', 'votes'])


In [115]:
print("Shape of dataframe: ", (zomato.count(), len(zomato.columns)))
print("Columns in dataframe: ", zomato.columns)


Shape of dataframe:  (16186, 12)
Columns in dataframe:  ['name', 'online_order', 'book_table', 'rate', 'location', 'cuisines', 'cost', 'reviews_list', 'city', 'Mean Rating', 'Scaled Mean Rating', 'top_words']


In [116]:
from pyspark.sql.functions import rand

df_percent = zomato.orderBy(rand()).limit(int(zomato.count() * 0.5))

In [117]:
print("Shape of dataframe: ", (df_percent.count(), len(df_percent.columns)))


Shape of dataframe:  (8093, 12)


In [123]:
df_percent = df_percent.withColumn("name", F.col("name").cast("string"))
df_percent = df_percent.select("*").orderBy("name")
df_percent.createOrReplaceTempView("df_percent")
df_percent = spark.sql("SELECT * FROM df_percent ORDER BY name")
df_percent = df_percent.withColumnRenamed("name", "index").withColumnRenamed("percent", "value")


In [124]:
from pyspark.sql.functions import col

indices = df_percent.select(col("index")).rdd.flatMap(lambda x: x).collect()

In [125]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

tokenizer = Tokenizer(inputCol="reviews_list", outputCol="words")
wordsData = tokenizer.transform(df_percent)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
tfidf_matrix = idfModel.transform(featurizedData).select("features")

In [126]:
from pyspark.ml.feature import Normalizer

normalizer = Normalizer(inputCol="features", outputCol="norm")
data = normalizer.transform(tfidf_matrix)

cosine_similarities = data.rdd.cartesian(data.rdd).map(lambda x: (x[0][0], x[1][0], float(x[0][1].dot(x[1][1])))).toDF(["id1", "id2", "similarity"])

In [161]:
from pyspark.sql.functions import col

def recommend(name, cosine_similarities=cosine_similarities):
    recommend_restaurant = []
    idx = indices.index(name)
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    top30_indexes = list(score_series.iloc[0:31].index)
    
    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])
    
    df_new = spark.createDataFrame([], ['cuisines', 'Mean Rating', 'cost'])
    
    for each in recommend_restaurant:
        df_new = df_new.union(df_percent.select(['cuisines','Mean Rating', 'cost']).where(col('name') == each).sample(False, 1.0))
    
    df_new = df_new.dropDuplicates(['cuisines','Mean Rating', 'cost'])
    df_new = df_new.orderBy(col('Mean Rating').desc()).limit(10)
    
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(df_new.count()), name))
    
    return df_new


In [ ]:
recommend('Pai Vihar')